In [6]:
from jqdatasdk import *
import akshare as ak
import baostock as bs
import pandas as pd
import re
import datetime
import time

In [ ]:
# 登录JoinQuant
from jqdatasdk import bond
auth("account", "password")  # Requires joinquant account.

# 查看当日剩余检索的数据行数
print(f"Remaining daily data queries allowed on JoinQuant: {get_query_count()}")

In [ ]:
# 登陆baostock系统
lg = bs.login()

# 显示登陆返回信息
print("login respond error_code:" + lg.error_code)
print("login respond error_msg:" + lg.error_msg)

# 利率等数据的获取与设置

## 模型参数设置

In [14]:
# 初始时间设置
startDate, endDate = "19950101", "20211231"  # 时间区间

# 检查时间区间是否有效
if int(startDate) > int(endDate) or int(endDate) > datetime.date.today().year * 10000 + \
        datetime.date.today().month * 100 + datetime.date.today().day:
    print("Invalid Time Interval")
    quit()

In [ ]:
# 其它初始模型参数设置(其实在第一个文件中用不到)
tradePercent = 0.1  # 多空比例
laggedPeriod = pd.Timedelta("30 D")  # 从今天倒推不计入windowPeriod的时间
windowPeriod = pd.Timedelta(str(30 * 11) + " D")  # 回溯期，以计算累计收益率
holdPeriod = pd.Timedelta("30 D")  # 持有新投资组合的时间

## 根据算法设置无风险利率

| 时间(t) | 数据来源 |
| :--- | ----: |
| t $\leq$ 2002-08-06 | 三个月期定期银行存款利率 |
| 2002-08-07 $\leq$ t $\leq$ 2006-10-07 | 三个月期中央银行票据的票面利率 |
| 2006-10-08 $\leq$ t | 上海银行间三个月同业拆放利率 |


In [9]:
# 定义函数输出利率数据库所需的日期格式
def interest_dateformat(date: str) -> str:
    return f"{date[:4]}-{date[4:6]}-{date[6:]}"

### 三个月定期银行存款利率

In [11]:
# 获取银行定期存款利率
rs = bs.query_deposit_rate_data(start_date = interest_dateformat(str(int(startDate) - 50000)),
                                end_date = interest_dateformat(endDate))
print("query_deposit_rate_data respond error_code:" + rs.error_code)
print("query_deposit_rate_data respond error_msg:" + rs.error_msg)

# deposit_rate结果集
interest_data_list = []
while (rs.error_code == "0") & rs.next():
    # 获取一条记录，将记录合并在一起
    interest_data_list.append(rs.get_row_data())
result = pd.DataFrame(interest_data_list, columns=rs.fields)

# 结果集输出到csv文件
result.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_data\\deposit_interest_rate.csv",
              encoding = "gbk", index = False)
print(result)

query_deposit_rate_data respond error_code:0
query_deposit_rate_data respond  error_msg:success
       pubDate demandDepositRate fixedDepositRate3Month  \
0   1995-01-01                                            
1   1995-07-01                                            
2   1996-05-01          2.970000               4.860000   
3   1996-08-23          1.980000               3.330000   
4   1997-10-23          1.710000               2.880000   
5   1998-03-25          1.710000               2.880000   
6   1998-07-01          1.440000               2.790000   
7   1998-12-07          1.440000               2.790000   
8   1999-06-10          0.990000               1.980000   
9   2002-02-21          0.720000               1.710000   
10  2004-10-29          0.720000               1.710000   
11  2006-04-28                                            
12  2006-08-19          0.720000               1.800000   
13  2007-03-18          0.720000               1.980000   
14  2007-05-19     

### 三个月期中央银行票据的票面利率


In [ ]:
# 获取央行票据数据
central_bank_bill = bond.run_query(query(bond.BOND_BASIC_INFO).filter(
    bond.BOND_BASIC_INFO.bond_type_id == "703019").limit(5000))

# 求出每一笔央行票据的期限
central_bank_bill["maturity"] = central_bank_bill["maturity_date"] - central_bank_bill["interest_begin_date"]

# 过滤出期限为三个月的央行票据
central_bank_bill = central_bank_bill[("80d" < central_bank_bill["maturity"]) &
                                      (central_bank_bill["maturity"] < "100d")]

# 结果集输出到csv文件
central_bank_bill.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_data\\central_bank_bill.csv",
                         encoding="gbk", index=False)

### 上海银行间三个月同业拆放利率 (Shibor)

In [13]:
# 获取银行间同业拆放利率
rs = bs.query_shibor_data(start_date = interest_dateformat(startDate),
                          end_date = interest_dateformat(endDate))
print("query_shibor_data respond error_code:" + rs.error_code)
print("query_shibor_data respond  error_msg:" + rs.error_msg)

# shibor结果集
interest_data_list = []
while (rs.error_code == "0") & rs.next():
    # 获取一条记录，将记录合并在一起
    interest_data_list.append(rs.get_row_data())
result = pd.DataFrame(interest_data_list, columns = rs.fields)

# 结果集输出到csv文件
result.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_data\\shibor.csv", 
              encoding = "gbk", index = False)
print(result)

query_shibor_data respond error_code:0
query_shibor_data respond  error_msg:success
            date  shiborON  shibor1W  shibor2W  shibor1M  shibor3M  shibor6M  \
0     2006-10-08  2.118400  2.293000  2.384800  2.531900  2.611000  2.740400   
1     2006-10-09  2.099000  2.296000  2.397200  2.552200  2.624800  2.743100   
2     2006-10-10  2.092200  2.297100  2.423600  2.573900  2.632500  2.745400   
3     2006-10-11  2.095500  2.293200  2.493000  2.586400  2.633800  2.747500   
4     2006-10-12  2.094300  2.290400  2.524000  2.590000  2.638000  2.747000   
...          ...       ...       ...       ...       ...       ...       ...   
3759  2021-10-25  1.610000  2.243000  2.410000  2.380000  2.438000  2.522000   
3760  2021-10-26  1.546000  2.262000  2.424000  2.387000  2.443000  2.525000   
3761  2021-10-27  1.921000  2.268000  2.441000  2.392000  2.445000  2.529000   
3762  2021-10-28  1.940000  2.301000  2.441000  2.396000  2.448000  2.533000   
3763  2021-10-29  2.142000  2.299000

# 沪深300成分股 / A股成分股 获取与简单预处理

In [14]:
rs = bs.query_hs300_stocks()
print("query_hs300 error_code:" + rs.error_code)
print("query_hs300 error_msg:" + rs.error_msg)

# 沪深300成分股结果集
hs300_stocks = []
while (rs.error_code == "0") & rs.next():
    # 获取一条记录，将记录合并在一起
    hs300_stocks.append(rs.get_row_data())
result = pd.DataFrame(hs300_stocks, columns=rs.fields)

# 结果集输出到csv文件
result.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\data\\hs300_stocks.csv", 
              encoding="gbk", index=False)
print(result)

query_hs300 error_code:0
query_hs300  error_msg:success
     updateDate       code code_name
0    2022-01-17  sh.600000      浦发银行
1    2022-01-17  sh.600009      上海机场
2    2022-01-17  sh.600010      包钢股份
3    2022-01-17  sh.600011      华能国际
4    2022-01-17  sh.600015      华夏银行
..          ...        ...       ...
295  2022-01-17  sz.300782       卓胜微
296  2022-01-17  sz.300866      安克创新
297  2022-01-17  sz.300888      稳健医疗
298  2022-01-17  sz.300896       爱美客
299  2022-01-17  sz.300999       金龙鱼

[300 rows x 3 columns]


In [ ]:
# 登出baostock系统
bs.logout()

# 设置使用HS300指数还是A股 !

In [20]:
# 得到所有沪深300成分股或A股成分股的代码
def GetCodeLst(fromWhat: str) -> list:
    if fromWhat == "HS300":
        hs300_Stocks = pd.read_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_data\\hs300_stocks.csv",
                                   encoding="gbk").set_index("code")
        return list(map(lambda x: re.search(pattern="[0-9]+", string=x).group(), 
                        list(hs300_Stocks.index)))

    elif fromWhat == "A":
        return list(ak.stock_info_sh_name_code(indicator="主板A股")["代码"]) + \
               list(ak.stock_info_sh_name_code(indicator="科创板")["代码"]) + \
               list(ak.stock_info_sz_name_code(indicator="A股列表")["A股代码"])

In [23]:
codeLst = GetCodeLst(fromWhat="HS300")

300

In [16]:
# 获取交易日历
def GetTradeCalender(start: str, end: str) -> pd.Series:
    cal = ak.tool_trade_date_hist_sina()
    return cal["trade_date"][
        (datetime.date(int(start[:4]),int(start[4:6]), int(start[6:])) <= cal["trade_date"]) & 
        (cal["trade_date"] <= datetime.date(int(end[:4]), int(end[4:6]), int(end[6:])))]

calender = GetTradeCalender(startDate, endDate)

# 保存交易日历
calender.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_data\\calender.csv", index=True, header=True)

In [18]:
# 创建初始的收盘价、涨跌幅、市净率、总市值的DataFrame
close_df, return_df, BM_df, MV_df = pd.DataFrame(index=calender), pd.DataFrame(index=calender), \
                                    pd.DataFrame(index=calender), pd.DataFrame(index=calender)

""
trade_date
1995-01-03
1995-01-04
1995-01-05
1995-01-06
1995-01-09
...
2021-12-27
2021-12-28
2021-12-29


# 将每支个股的信息录入各DataFrame中

In [ ]:
# 用以将日期格式转换为datetime.date
def ParseDate(date: str) -> datetime.date:
    date = list(map(int, re.findall(pattern="[0-9]+", string=str(date))))
    return datetime.date(date[0], date[1], date[2])

In [ ]:
# what_now 代表当前在 while loop 中的位置，exceptionLst 包含无法按照规定交易日index进行合并的股票代码，
# failure 代表通过 api 获取某股票数据连续失败的次数，maximum_failure_allowed 代表允许连续失败的最大次数。
what_now, exceptionLst, failure, maximum_failure_allowed, length = 0, [], 0, 3, len(codeLst)
print(f"Data of {length} stocks in total need to be collected, waiting......")
while what_now < length:
    code = codeLst[what_now]

    try:
        # 获取个股自 1995-01-01 至 2021-12-31 的收盘价与涨跌幅(日频)
        this_stock_hist_daily = ak.stock_zh_a_hist(symbol=code, period="daily",
                                start_date=startDate, end_date=endDate,
                                adjust="hfq")[["日期", "收盘", "涨跌幅"]].set_index("日期")

        # 获取个股自 1995-01-01 至 2021-12-31 的市净率与总市值(日频)
        this_stock_BMMV_daily = \
            ak.stock_a_lg_indicator(symbol=code)[["trade_date", "pb", "total_mv"
                                                 ]].set_index("trade_date")

        # index格式全部转换为datetime
        this_stock_hist_daily.index = map(ParseDate, list(this_stock_hist_daily.index))
        this_stock_BMMV_daily.index = map(ParseDate, list(this_stock_BMMV_daily.index))

        # (获取数据)成功则清零failure
        failure = 0

        # 尝试合并数据
        try:
            close_df[code] = this_stock_hist_daily["收盘"]
            return_df[code] = this_stock_hist_daily["涨跌幅"]
            BM_df[code] = 1 / this_stock_BMMV_daily["pb"]  # 账面市值比BM与市净率互为倒数
            MV_df[code] = this_stock_BMMV_daily["total_mv"]

            # 报告数据获取且合并成功
            print(f"{what_now}/{length}. Data collected and merged for code: {code}")

        except:
            # 报告数据合并失败
            print(f"{what_now}/{length}. Met an unknown error when merging data of code: {code}")
            exceptionLst.append((what_now, code))

        # 每获取100组数据，就及时写入csv临时文件
        if what_now % 100 == 0:
            # 写临时文件
            close_df.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_temp_data\\close_temp.csv", 
                            index=True, header=True)
            return_df.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_temp_data\\return_temp.csv", 
                             index=True, header=True)
            BM_df.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_temp_data\\BM_temp.csv", 
                         index=True, header=True)
            MV_df.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_temp_data\\MV_temp.csv", 
                         index=True, header=True)
            
            pd.Series(exceptionLst).to_csv(
                "C:\\Users\\asus\\Desktop\\Carhart\\HS300_temp_data\\exceptionLst.csv",
                index=False, header=True)

            # 打印当前备份保存情况
            print(f"Temporary file is saved at: {code}. Position is: {what_now}")

        # 每30组数据(30 * 4375 * 4)暂停45秒，避免IP被封禁
        if what_now % 30 == 0:
            # 暂停45秒
            print("Resuming in 45 seconds......")
            time.sleep(45)

        what_now += 1

    except:
        # 打印当前错误位置或断点并反馈连续错误次数
        failure += 1
        print(f"{what_now}/{length}. Problem encountered at code: {code}. Failure = {failure}")

        if failure > maximum_failure_allowed:
            break  # 错误过多直接退出
        else:
            print(f"Retrying in {60 * failure} seconds......")
            time.sleep(60 * failure)  # 睡眠1-3分钟
            continue


# 若while loop未曾中断，则写入csv最终文件
else:
    close_df.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_data\\close.csv", index=True, header=True)
    return_df.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_data\\return.csv", index=True, header=True)
    BM_df.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_data\\BM.csv", index=True, header=True)
    MV_df.to_csv("C:\\Users\\asus\\Desktop\\Carhart\\HS300_data\\MV.csv", index=True, header=True)
    
    # 判断 exceptionLst 是否为空
    if len(exceptionLst) == 0:
        print("All data are collected and merged successfully")
    else:
        print("exceptionLst is not empty: failed to merge some data")